In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from specim_test.specim.specfuncs import spec2d
from astropy.io import fits as pf

In [2]:
file_loc = '../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_D/Science/*'

In [3]:
file_list = []
out_file = []
for i,p in enumerate(glob.glob(file_loc)):
    if 'spec2d' in p and 'MG0751' in p:

        file_list.append(p)
        frame = p.split('_')[-3][:4]
        out_file.append('MG0751_r'+ frame + '_mask_new.fits')

In [ ]:
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[30, 200])
    spec.set_dispaxis('y')
    spec.subtract_sky_2d(use_skymod=True)# outfile=out_file[i])
    #spec.szap(outfile=out_file[i], use_skymod=True)
    filled_data, mask = szap(spec.data)

In [4]:
spec1 = spec2d.Spec2d(file_list[0], hext=12, xtrim=[30, 200])
spec1.set_dispaxis('y')
spec1.subtract_sky_2d(use_skymod=True)
spec2 = spec2d.Spec2d(file_list[1], hext=12, xtrim=[30, 200])
spec2.set_dispaxis('y')
spec2.subtract_sky_2d(use_skymod=True)


Loading file ../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_D/Science/spec2d_r201213_0034-MG0751+2716_LRISr_2020Dec13T120424.557.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_D/Science/spec2d_r201213_0034-MG0751+2716_LRISr_2020Dec13T120424.557.fits (HDU=12)
The input dataset was trimmed
 xrange: 30:201.  yrange: 0:4096
Final data dimensions (x y): 171 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 171


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



pypeit generated sky model will be used for sky subtraction


From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


Loading file ../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_D/Science/spec2d_r201213_0035-MG0751+2716_LRISr_2020Dec13T124808.006.f

In [5]:
sp_1 = spec1.data[:, 30:201]
sp_2 = spec2.data[:, 30:201]

In [ ]:
%matplotlib notebook
count, bins, patch= plt.hist(sp_1.flatten(), bins=1000)
counts, bins1, patch = plt.hist(sp_2.flatten(), bins=1000)
#bins2= plt.hist(sp2.flatten(), bins=250)
#bins = plt.hist((sp2-sp1).flatten(), bins=200)
plt.yscale('log')

In [6]:
#sp_2 = pf.open('B0445_median_image.fits')[0].data
mask = np.zeros(sp_1.shape, dtype=bool)
#print(sp_1.shape)

for k in range(0, sp_1.shape[0], 400):
    for i in range(0, sp_1.shape[1], 5):
        sp1 = sp_1[k:k+400, i:i+5]
        sp2 = sp_2[k:k+400, i:i+5]
        count1, bins1 = np.histogram(sp1.flatten(), bins=1000)
        count2, bins2 = np.histogram(sp2.flatten(), bins=1000)
        sm1 = 0
        sm2 = 0
        cnt1 = 0
        cnt2 = 0
        tot_pix =sp1.flatten().shape[0]

        for j,p in enumerate(count1):
            if p > cnt1:
                cnt1 = p
                max_bin1 = bins1[j+1]
            sm1 += p
            if sm1/tot_pix >= 0.8 :
                border1 = 0.5*(bins1[j+1] + bins1[j])
                break
        #print(border)
        for j,p in enumerate(count2):
            if p > cnt2:
                cnt2 = p
                max_bin2 = 0.5*(bins2[j+1] + bins2[j])
            sm2 += p
            if sm2/tot_pix >= 0.8 :
                border2 = bins2[j+1]
                break

        a = np.where(sp1>border1)
        a =np.transpose(a)

        if abs(border1 - max_bin1) < 50 :
            mp = 100    #/ abs(border1 - max_bin1)
        elif abs(border1 - max_bin1) < 100 :
            mp = 200
        elif abs(border1 - max_bin1) < 300:
            mp = 300
        else:
            mp = 400

        for j, p in enumerate(a):

            #print(sp2[p[0]][p[1]])
            if sp2[p[0]][p[1]] >0 :
                #pass 
                if sp1[p[0]][p[1]] > (sp2[p[0]][p[1]]+mp) :
                    mask[k+p[0]][i+p[1]] =1 
            else:
                if sp1[p[0]][p[1]] > 3*border1 :
                    mask[k+p[0]][i+p[1]] =1

#print(mask)
#sp_1[mask] = sp_2[mask]

#return sp_1, mask.astype(int)

In [ ]:
sp_1[mask] = -50000

In [7]:
mf = np.transpose(np.where(mask==1))
fill = np.zeros(sp_1.shape)
#print(mf.shape[0])
for i,p in enumerate(mf):
    #mask[p[0]+rw_st][p[1]+clm_st] = 1
    reg = sp_1[p[0]- 20 : p[0]+ 21, p[1]:p[1]+1]
    fill[p[0]][p[1]] = np.median(reg)
sp_1[mask] = fill[mask]

In [8]:
pf.PrimaryHDU(sp_1.T).writeto('sp1_fill.fits')

In [ ]:
sp_1.shape

In [ ]:
np.transpose(np.where(mask==1))

In [ ]:
sp_1.shape